In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 250
num_epochs = 200
learning_rate = 1e-5
samples = 30000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (32, 32), color = 'white')
        row,col,ch= np.shape(img)
        mean = random.randint(1,6)
        var = random.random()
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2_pos1))

        img_1 = Image.new('RGB', (32, 32), color = 'white')
        row_1,col_1,ch_1= np.shape(img_1)
        mean_1 = random.randint(1,6)
        var_1 = random.random()
        sigma_1 = var_1**0.5
        gauss_pos2 = np.random.normal(mean_1,sigma_1,(row_1,col_1,ch_1))
        noisy_pos2 = img_1 + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw_1 = ImageDraw.Draw(im2_pos2)
        draw_1.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data2.append(np.asarray(im2_pos2))
    elif switch==1:
        img_blank = Image.new('RGB', (32,32), color = 'white')
        row_blank,col_blank,ch_blank= np.shape(img_blank)
        mean_blank = random.randint(1,6)
        var_blank = random.random()
        sigma_blank = var_blank**0.5
        gauss_blank_pos1 = np.random.normal(mean_blank,sigma_blank,(row_blank,col_blank,ch_blank))
        noisy_blank_pos1 = img_blank + gauss_blank_pos1
        im2_blank_pos1 = Image.fromarray(noisy_blank_pos1,'RGB')
        data1.append(np.asarray(im2_blank_pos1))

        img_blank_1 = Image.new('RGB', (32, 32), color = 'white')
        row_blank_1,col_blank_1,ch_blank_1= np.shape(img_blank_1)
        mean_blank_1 = random.randint(1,6)
        var_blank_1 = random.random()
        sigma_blank_1 = var_blank_1**0.5
        gauss_blank_pos2 = np.random.normal(mean_blank_1,sigma_blank_1,(row_blank_1,col_blank_1,ch_blank_1))
        noisy_blank_pos2 = img_blank_1 + gauss_blank_pos2
        im2_blank_pos2 = Image.fromarray(noisy_blank_pos2,'RGB')
        data2.append(np.asarray(im2_blank_pos2))



#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (32, 32), color = 'white')
row,col,ch= np.shape(img)
mean = random.randint(1,6)
var = random.random()
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b=b[:,:,0]
b = torch.from_numpy(b)
print(np.shape(b))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = y0[:,:,:,0]
y1 = y1[:,:,:,0]

print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=64
        )
        self.encoder_output_layer_1 = nn.Linear(
            in_features=64, out_features=32
        )
        self.encoder_output_layer_2 = nn.Linear(
            in_features=32, out_features=16
        )
        self.encoder_output_layer_3 = nn.Linear(
            in_features=16, out_features=8
        )
        self.encoder_output_layer_4 = nn.Linear(
            in_features=8, out_features=1
        )




    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        #activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        #code = torch.relu(code)
        code = self.encoder_output_layer_1(code)
        #code = torch.relu(code)
        code = self.encoder_output_layer_2(code)
        #code = torch.relu(code)
        code = self.encoder_output_layer_3(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_4(code)
        #code = torch.sigmoid(code)
        return code

#  use gpu if available

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=1024)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)



for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        x = x.view(-1, 1024)
        y = y.view(-1, 1024)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    b = b.view(-1,1024)
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print(np.shape(x[0]))
    print("x: ", model(x[0]))
    print("y:", model(y[0]))
    print("b:", model(b))



torch.Size([32, 32])
(30000, 32, 32)
epoch [1/200], loss:0.7682
torch.Size([1024])
x:  tensor([4.9934], grad_fn=<AddBackward0>)
y: tensor([5.3536], grad_fn=<AddBackward0>)
b: tensor([[3.7228]], grad_fn=<AddmmBackward>)
epoch [2/200], loss:0.2245
torch.Size([1024])
x:  tensor([4.9158], grad_fn=<AddBackward0>)
y: tensor([3.6594], grad_fn=<AddBackward0>)
b: tensor([[3.7736]], grad_fn=<AddmmBackward>)
epoch [3/200], loss:0.1482
torch.Size([1024])
x:  tensor([4.6822], grad_fn=<AddBackward0>)
y: tensor([4.1073], grad_fn=<AddBackward0>)
b: tensor([[3.7816]], grad_fn=<AddmmBackward>)
epoch [4/200], loss:-0.1177
torch.Size([1024])
x:  tensor([3.6905], grad_fn=<AddBackward0>)
y: tensor([4.3894], grad_fn=<AddBackward0>)
b: tensor([[3.8945]], grad_fn=<AddmmBackward>)
epoch [5/200], loss:-0.2118
torch.Size([1024])
x:  tensor([4.8599], grad_fn=<AddBackward0>)
y: tensor([4.9727], grad_fn=<AddBackward0>)
b: tensor([[4.0778]], grad_fn=<AddmmBackward>)
epoch [6/200], loss:-0.4139
torch.Size([1024])
x:  

epoch [46/200], loss:-1.0000
torch.Size([1024])
x:  tensor([5.8504], grad_fn=<AddBackward0>)
y: tensor([6.3456], grad_fn=<AddBackward0>)
b: tensor([[4.0773]], grad_fn=<AddmmBackward>)
epoch [47/200], loss:-1.0000
torch.Size([1024])
x:  tensor([5.8433], grad_fn=<AddBackward0>)
y: tensor([6.3283], grad_fn=<AddBackward0>)
b: tensor([[4.0751]], grad_fn=<AddmmBackward>)
epoch [48/200], loss:-1.0000
torch.Size([1024])
x:  tensor([5.8404], grad_fn=<AddBackward0>)
y: tensor([6.3319], grad_fn=<AddBackward0>)
b: tensor([[4.0705]], grad_fn=<AddmmBackward>)
epoch [49/200], loss:-1.0000
torch.Size([1024])
x:  tensor([4.0772], grad_fn=<AddBackward0>)
y: tensor([4.0763], grad_fn=<AddBackward0>)
b: tensor([[4.0756]], grad_fn=<AddmmBackward>)
epoch [50/200], loss:-1.0000
torch.Size([1024])
x:  tensor([4.0649], grad_fn=<AddBackward0>)
y: tensor([4.0702], grad_fn=<AddBackward0>)
b: tensor([[4.0708]], grad_fn=<AddmmBackward>)
epoch [51/200], loss:-0.9999
torch.Size([1024])
x:  tensor([5.8575], grad_fn=<Ad

epoch [91/200], loss:-0.9992
torch.Size([1024])
x:  tensor([3.5271], grad_fn=<AddBackward0>)
y: tensor([3.5327], grad_fn=<AddBackward0>)
b: tensor([[3.5377]], grad_fn=<AddmmBackward>)
epoch [92/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.5093], grad_fn=<AddBackward0>)
y: tensor([3.5041], grad_fn=<AddBackward0>)
b: tensor([[3.5101]], grad_fn=<AddmmBackward>)
epoch [93/200], loss:-0.9999
torch.Size([1024])
x:  tensor([5.2668], grad_fn=<AddBackward0>)
y: tensor([5.7563], grad_fn=<AddBackward0>)
b: tensor([[3.4791]], grad_fn=<AddmmBackward>)
epoch [94/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.4306], grad_fn=<AddBackward0>)
y: tensor([3.4402], grad_fn=<AddBackward0>)
b: tensor([[3.4436]], grad_fn=<AddmmBackward>)
epoch [95/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.4138], grad_fn=<AddBackward0>)
y: tensor([3.4077], grad_fn=<AddBackward0>)
b: tensor([[3.4143]], grad_fn=<AddmmBackward>)
epoch [96/200], loss:-0.9999
torch.Size([1024])
x:  tensor([5.1542], grad_fn=<Ad

epoch [136/200], loss:-0.9998
torch.Size([1024])
x:  tensor([2.3110], grad_fn=<AddBackward0>)
y: tensor([2.3119], grad_fn=<AddBackward0>)
b: tensor([[2.3062]], grad_fn=<AddmmBackward>)
epoch [137/200], loss:-0.9999
torch.Size([1024])
x:  tensor([4.0432], grad_fn=<AddBackward0>)
y: tensor([4.5341], grad_fn=<AddBackward0>)
b: tensor([[2.2827]], grad_fn=<AddmmBackward>)
epoch [138/200], loss:-1.0000
torch.Size([1024])
x:  tensor([2.2552], grad_fn=<AddBackward0>)
y: tensor([2.2569], grad_fn=<AddBackward0>)
b: tensor([[2.2573]], grad_fn=<AddmmBackward>)
epoch [139/200], loss:-0.9999
torch.Size([1024])
x:  tensor([2.2917], grad_fn=<AddBackward0>)
y: tensor([2.2819], grad_fn=<AddBackward0>)
b: tensor([[2.2847]], grad_fn=<AddmmBackward>)
epoch [140/200], loss:-0.9981
torch.Size([1024])
x:  tensor([2.2790], grad_fn=<AddBackward0>)
y: tensor([2.2843], grad_fn=<AddBackward0>)
b: tensor([[2.2944]], grad_fn=<AddmmBackward>)
epoch [141/200], loss:-0.9999
torch.Size([1024])
x:  tensor([2.2339], grad_

epoch [181/200], loss:-0.9995
torch.Size([1024])
x:  tensor([3.7177], grad_fn=<AddBackward0>)
y: tensor([4.1921], grad_fn=<AddBackward0>)
b: tensor([[1.9238]], grad_fn=<AddmmBackward>)
epoch [182/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.7171], grad_fn=<AddBackward0>)
y: tensor([4.1822], grad_fn=<AddBackward0>)
b: tensor([[1.9329]], grad_fn=<AddmmBackward>)
epoch [183/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.7586], grad_fn=<AddBackward0>)
y: tensor([4.2203], grad_fn=<AddBackward0>)
b: tensor([[1.9719]], grad_fn=<AddmmBackward>)
epoch [184/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.7026], grad_fn=<AddBackward0>)
y: tensor([4.1794], grad_fn=<AddBackward0>)
b: tensor([[1.9314]], grad_fn=<AddmmBackward>)
epoch [185/200], loss:-1.0000
torch.Size([1024])
x:  tensor([3.5943], grad_fn=<AddBackward0>)
y: tensor([4.0578], grad_fn=<AddBackward0>)
b: tensor([[1.8127]], grad_fn=<AddmmBackward>)
epoch [186/200], loss:-0.9999
torch.Size([1024])
x:  tensor([3.5817], grad_

In [103]:
data3=[]

for i in range(5):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = random.randint(1,6)
    var = random.random()
    sigma = var**0.5
    gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
    noisy_pos1 = img + gauss_pos1
    im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
    draw = ImageDraw.Draw(im2_pos1)
    draw.rectangle([16,16,24,24],fill='red',outline='red')
    data3.append(np.asarray(im2_pos1))

    y3 = np.array(data3, dtype=np.float32)
    #y1 = np.array(data2, dtype=np.float32)
    y3 = y3[:,:,:,0]
    #y1 = y1[:,:,:,0]
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].view(-1,784)))

x:  tensor([[0.7089]], grad_fn=<AddmmBackward>)
x:  tensor([[0.7069]], grad_fn=<AddmmBackward>)
x:  tensor([[0.7121]], grad_fn=<AddmmBackward>)
x:  tensor([[0.7019]], grad_fn=<AddmmBackward>)
x:  tensor([[0.7127]], grad_fn=<AddmmBackward>)
